# Unit 10: Data Cleaning - Practical Examples

## 📚 Overview

In this practical notebook, you'll learn to clean messy real-world data:
- Remove duplicates from customer data
- Fix inconsistent category labels
- Handle incorrect data types
- Parse various date formats
- Clean text and standardize values
- Build complete cleaning pipelines

**Learning Objectives:**
1. Identify common data quality issues
2. Apply systematic cleaning techniques
3. Validate cleaning results
4. Build reusable cleaning functions

---

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import re

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

---

## Part 1: Removing Duplicates

Let's work with a messy customer dataset containing various types of duplicates.

In [ ]:
# Create messy customer data with duplicates
np.random.seed(42)

customers = pd.DataFrame({
    'customer_id': [1001, 1002, 1003, 1001, 1004, 1002, 1005, 1003, 1006, 1007, 1001],
    'name': ['John Smith', 'Jane Doe', 'Bob Wilson', 'John Smith', 'Alice Brown',
             'Jane Doe', 'Charlie Davis', 'Bob Wilson', 'Emma Johnson', 'David Lee', 'John Smith'],
    'email': ['john@email.com', 'jane@email.com', 'bob@email.com', 'john@email.com',
              'alice@email.com', 'jane.doe@email.com', 'charlie@email.com', 'bob@email.com',
              'emma@email.com', 'david@email.com', 'john.smith@email.com'],
    'phone': ['555-1234', '555-5678', '555-9012', '555-1234', '555-3456',
              '555-5678', '555-7890', '555-9012', '555-2468', '555-1357', '555-1234'],
    'signup_date': ['2024-01-15', '2024-01-20', '2024-01-25', '2024-02-01', '2024-02-10',
                   '2024-02-15', '2024-03-01', '2024-03-05', '2024-03-10', '2024-03-15', '2024-03-20'],
    'total_purchases': [5, 3, 7, 8, 2, 4, 6, 9, 1, 4, 12]
})

print(f"Dataset shape: {customers.shape}")
print(f"\nFirst few rows:")
customers

In [ ]:
# Step 1: Detect duplicates
print("=" * 60)
print("DUPLICATE DETECTION")
print("=" * 60)

# Check for exact duplicates
print(f"\nExact duplicate rows: {customers.duplicated().sum()}")

# Check duplicates based on customer_id
print(f"Duplicate customer_ids: {customers.duplicated(subset=['customer_id']).sum()}")

# View all duplicate records
print(f"\nDuplicate records (based on customer_id):")
duplicates = customers[customers.duplicated(subset=['customer_id'], keep=False)]
print(duplicates.sort_values('customer_id'))

In [ ]:
# Step 2: Investigate duplicate patterns
print("\nDuplicate Analysis:")
duplicate_ids = customers[customers.duplicated(subset=['customer_id'], keep=False)]

for cust_id in duplicate_ids['customer_id'].unique():
    print(f"\nCustomer ID {cust_id}:")
    print(customers[customers['customer_id'] == cust_id][['name', 'email', 'signup_date', 'total_purchases']])

In [ ]:
# Step 3: Remove duplicates - Keep most recent with highest purchases
print("Strategy: Keep the record with the most recent signup and highest purchases\n")

# Convert signup_date to datetime for sorting
customers['signup_date'] = pd.to_datetime(customers['signup_date'])

# Sort by customer_id, signup_date (descending), and total_purchases (descending)
customers_sorted = customers.sort_values(
    ['customer_id', 'signup_date', 'total_purchases'],
    ascending=[True, False, False]
)

# Keep first occurrence (most recent, highest purchases)
customers_clean = customers_sorted.drop_duplicates(subset=['customer_id'], keep='first')

print(f"Original rows: {len(customers)}")
print(f"After removing duplicates: {len(customers_clean)}")
print(f"Rows removed: {len(customers) - len(customers_clean)}")

print("\nCleaned dataset:")
customers_clean.sort_values('customer_id')

In [ ]:
# Verify no duplicates remain
print("Verification:")
print(f"Remaining duplicates: {customers_clean.duplicated(subset=['customer_id']).sum()}")
print("✅ All duplicates removed!" if customers_clean.duplicated(subset=['customer_id']).sum() == 0 else "⚠️ Duplicates still exist")

---

## Part 2: Fixing Inconsistent Labels

Let's clean a product dataset with inconsistent category names.

In [ ]:
# Create messy product data
products = pd.DataFrame({
    'product_id': range(1, 21),
    'product_name': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones',
                    'Webcam', 'Speakers', 'Tablet', 'Phone', 'Charger',
                    'Cable', 'Stand', 'Bag', 'Dock', 'Adapter',
                    'Router', 'Printer', 'Scanner', 'Camera', 'Microphone'],
    'category': ['Electronics', 'electronics', 'ELECTRONICS', 'Electronics ',
                'Audio', 'electronics', 'Audio', 'Electronics', 'Electronics', 'Accessories',
                'accessories', 'Accessories ', 'Accessories', 'Electronics', 'accessories',
                'Network', 'Office', 'office', 'Electronics', 'Audio'],
    'status': ['Active', 'active', 'ACTIVE', 'A', 'Active',
              'Inactive', 'I', 'Active', 'active', 'Active',
              'Discontinued', 'Active', 'I', 'Active', 'inactive',
              'Active', 'A', 'Inactive', 'Active', 'ACTIVE'],
    'price': [999.99, 25.50, 75.00, 299.99, 150.00,
             89.99, 120.00, 599.99, 799.99, 29.99,
             15.99, 45.00, 79.99, 199.99, 19.99,
             129.99, 399.99, 299.99, 899.99, 199.99]
})

print("Messy Product Dataset:")
print(products.head(10))

In [ ]:
# Inspect inconsistent labels
print("=" * 60)
print("LABEL INCONSISTENCIES")
print("=" * 60)

print("\nUnique Category values:")
print(products['category'].unique())
print(f"\nTotal unique categories: {products['category'].nunique()}")

print("\nCategory value counts:")
print(products['category'].value_counts())

print("\n" + "="*60)
print("\nUnique Status values:")
print(products['status'].unique())
print(f"\nTotal unique statuses: {products['status'].nunique()}")

print("\nStatus value counts:")
print(products['status'].value_counts())

In [ ]:
# Clean categories - Step by step
products_clean = products.copy()

# Step 1: Strip whitespace
products_clean['category'] = products_clean['category'].str.strip()
print("After stripping whitespace:")
print(products_clean['category'].unique())

# Step 2: Standardize case (Title Case)
products_clean['category'] = products_clean['category'].str.title()
print("\nAfter title case:")
print(products_clean['category'].unique())

print(f"\nReduced from {products['category'].nunique()} to {products_clean['category'].nunique()} unique categories")

In [ ]:
# Clean status using mapping
status_mapping = {
    'A': 'Active',
    'I': 'Inactive',
    'active': 'Active',
    'ACTIVE': 'Active',
    'inactive': 'Inactive',
    'INACTIVE': 'Inactive',
    'Discontinued': 'Inactive'  # Map discontinued to inactive
}

# Apply mapping, keep original if not in mapping
products_clean['status'] = products_clean['status'].replace(status_mapping)

print("After standardizing status:")
print(products_clean['status'].unique())
print("\nStatus counts:")
print(products_clean['status'].value_counts())

print(f"\nReduced from {products['status'].nunique()} to {products_clean['status'].nunique()} unique statuses")

In [ ]:
# Visualize before and after
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Before - Categories
products['category'].value_counts().plot(kind='barh', ax=axes[0, 0], color='salmon')
axes[0, 0].set_title('Categories - Before Cleaning', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Count')

# After - Categories
products_clean['category'].value_counts().plot(kind='barh', ax=axes[0, 1], color='lightgreen')
axes[0, 1].set_title('Categories - After Cleaning', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Count')

# Before - Status
products['status'].value_counts().plot(kind='bar', ax=axes[1, 0], color='salmon')
axes[1, 0].set_title('Status - Before Cleaning', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=45)

# After - Status
products_clean['status'].value_counts().plot(kind='bar', ax=axes[1, 1], color='lightgreen')
axes[1, 1].set_title('Status - After Cleaning', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Count')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n✅ Labels successfully standardized!")

---

## Part 3: Handling Incorrect Data Types

Let's work with data that has incorrect data types.

In [ ]:
# Create dataset with type issues
sales_data = pd.DataFrame({
    'order_id': ['1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008'],
    'quantity': ['5', '3', '10', 'N/A', '7', '15', '2', '8'],
    'unit_price': ['$29.99', '$49.50', '$15.00', '$99.99', '$39.99', '$25.50', '$75.00', '$19.99'],
    'total': ['149.95', '148.50', '150.00', 'error', '279.93', '382.50', '150.00', '159.92'],
    'order_date': ['2024-01-15', '01/20/2024', 'Jan 25, 2024', '2024-02-01', '02/10/2024', 'Feb 15, 2024', '2024-03-01', '03/05/2024'],
    'shipped': ['Yes', 'No', 'Yes', 'Yes', '1', '0', 'True', 'False'],
    'customer_id': [101, 102, 103, 104, 105, 106, 107, 108]
})

print("Dataset with Type Issues:")
print(sales_data)
print("\nData Types:")
print(sales_data.dtypes)

In [ ]:
# Identify type issues
print("=" * 60)
print("TYPE ISSUES IDENTIFIED")
print("=" * 60)

print("\n1. order_id: Currently object, should be int")
print(f"   Sample: {sales_data['order_id'].iloc[0]} (type: {type(sales_data['order_id'].iloc[0])})")

print("\n2. quantity: Currently object, should be numeric (has 'N/A')")
print(f"   Sample: {sales_data['quantity'].iloc[0]} (type: {type(sales_data['quantity'].iloc[0])})")
print(f"   Invalid values: {sales_data[sales_data['quantity'] == 'N/A']['quantity'].tolist()}")

print("\n3. unit_price: Currently object, should be float (has '$' symbol)")
print(f"   Sample: {sales_data['unit_price'].iloc[0]} (type: {type(sales_data['unit_price'].iloc[0])})")

print("\n4. total: Currently object, should be float (has 'error')")
print(f"   Sample: {sales_data['total'].iloc[0]} (type: {type(sales_data['total'].iloc[0])})")

print("\n5. order_date: Currently object, should be datetime (multiple formats)")
print(f"   Formats found: {sales_data['order_date'].tolist()}")

print("\n6. shipped: Currently object, should be boolean (mixed Yes/No/1/0/True/False)")
print(f"   Values: {sales_data['shipped'].unique()}")

In [ ]:
# Fix data types - Step by step
sales_clean = sales_data.copy()

# 1. Convert order_id to int
sales_clean['order_id'] = pd.to_numeric(sales_clean['order_id'], errors='coerce').astype('Int64')
print("✅ order_id converted to Int64")

# 2. Clean and convert quantity
sales_clean['quantity'] = sales_clean['quantity'].replace('N/A', np.nan)
sales_clean['quantity'] = pd.to_numeric(sales_clean['quantity'], errors='coerce')
print("✅ quantity converted to numeric (N/A → NaN)")

# 3. Clean and convert unit_price (remove $ and convert)
sales_clean['unit_price'] = sales_clean['unit_price'].str.replace('$', '').str.replace(',', '')
sales_clean['unit_price'] = pd.to_numeric(sales_clean['unit_price'], errors='coerce')
print("✅ unit_price converted to float (removed $)")

# 4. Convert total (error → NaN)
sales_clean['total'] = pd.to_numeric(sales_clean['total'], errors='coerce')
print("✅ total converted to float ('error' → NaN)")

# 5. Convert order_date to datetime
sales_clean['order_date'] = pd.to_datetime(sales_clean['order_date'], errors='coerce')
print("✅ order_date converted to datetime")

# 6. Convert shipped to boolean
bool_mapping = {
    'Yes': True, 'yes': True, 'YES': True, 'Y': True, 'y': True,
    'No': False, 'no': False, 'NO': False, 'N': False, 'n': False,
    '1': True, 1: True, 'True': True, 'true': True,
    '0': False, 0: False, 'False': False, 'false': False
}
sales_clean['shipped'] = sales_clean['shipped'].map(bool_mapping)
print("✅ shipped converted to boolean")

print("\n" + "="*60)
print("CLEANED DATA TYPES")
print("="*60)
print(sales_clean.dtypes)

In [ ]:
# Verify and display cleaned data
print("Cleaned Dataset:")
print(sales_clean)

print("\nMissing values after conversion:")
print(sales_clean.isna().sum())

print("\n✅ All data types corrected successfully!")

In [ ]:
# Now we can perform calculations!
print("Now we can do calculations:")
print(f"\nTotal quantity sold: {sales_clean['quantity'].sum()}")
print(f"Average unit price: ${sales_clean['unit_price'].mean():.2f}")
print(f"Total revenue: ${sales_clean['total'].sum():.2f}")
print(f"Orders shipped: {sales_clean['shipped'].sum()} out of {len(sales_clean)}")

# Date operations
print(f"\nFirst order date: {sales_clean['order_date'].min().strftime('%Y-%m-%d')}")
print(f"Last order date: {sales_clean['order_date'].max().strftime('%Y-%m-%d')}")
print(f"Date range: {(sales_clean['order_date'].max() - sales_clean['order_date'].min()).days} days")

---

## Part 4: Parsing and Converting Dates

Let's work with various date formats and extract useful components.

In [ ]:
# Create dataset with various date formats
events = pd.DataFrame({
    'event_id': range(1, 16),
    'event_name': ['Conference', 'Workshop', 'Seminar', 'Webinar', 'Training',
                  'Meeting', 'Launch', 'Review', 'Planning', 'Demo',
                  'Presentation', 'Interview', 'Audit', 'Survey', 'Consultation'],
    'date_str': ['2024-01-15', '01/20/2024', 'Jan 25, 2024', '2024-02-01 14:30:00',
                '02/10/2024', 'February 15, 2024', '2024-03-01', '03/05/2024 09:00',
                'Mar 10, 2024', '2024-03-15', '20240320', '03/25/2024',
                'April 1, 2024', '2024-04-05', '04/10/2024']
})

print("Events with Various Date Formats:")
print(events)
print(f"\nCurrent type of date_str: {events['date_str'].dtype}")

In [ ]:
# Convert to datetime (pandas can handle multiple formats automatically)
events['event_date'] = pd.to_datetime(events['date_str'], errors='coerce')

print("After conversion:")
print(events[['event_name', 'date_str', 'event_date']])
print(f"\nNew type: {events['event_date'].dtype}")

# Check for conversion failures
failed_conversions = events[events['event_date'].isna()]
if len(failed_conversions) > 0:
    print(f"\n⚠️ Failed to convert {len(failed_conversions)} dates:")
    print(failed_conversions[['event_name', 'date_str']])
else:
    print("\n✅ All dates converted successfully!")

In [ ]:
# Extract date components
events['year'] = events['event_date'].dt.year
events['month'] = events['event_date'].dt.month
events['month_name'] = events['event_date'].dt.month_name()
events['day'] = events['event_date'].dt.day
events['day_of_week'] = events['event_date'].dt.dayofweek  # 0=Monday, 6=Sunday
events['day_name'] = events['event_date'].dt.day_name()
events['quarter'] = events['event_date'].dt.quarter
events['is_weekend'] = events['event_date'].dt.dayofweek.isin([5, 6])

print("Date Components Extracted:")
print(events[['event_name', 'event_date', 'month_name', 'day_name', 'quarter', 'is_weekend']].head(10))

In [ ]:
# Date calculations
today = pd.to_datetime('2024-04-15')

events['days_until_event'] = (events['event_date'] - today).dt.days
events['is_past'] = events['event_date'] < today
events['is_future'] = events['event_date'] > today

# Categorize events
events['time_category'] = pd.cut(
    events['days_until_event'],
    bins=[-np.inf, -30, 0, 30, np.inf],
    labels=['Long Past', 'Recent Past', 'Upcoming Soon', 'Future']
)

print("\nTime-based Analysis:")
print(events[['event_name', 'event_date', 'days_until_event', 'time_category']])

In [ ]:
# Visualize events over time
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Events by month
events['month_name'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Events by Month', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count')
axes[0].set_xlabel('Month')
axes[0].tick_params(axis='x', rotation=45)

# Events by day of week
events['day_name'].value_counts().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
).plot(kind='bar', ax=axes[1], color='lightcoral')
axes[1].set_title('Events by Day of Week', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Count')
axes[1].set_xlabel('Day')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\nWeekend events: {events['is_weekend'].sum()}")
print(f"Weekday events: {(~events['is_weekend']).sum()}")

---

## Part 5: String Cleaning and Normalization

Let's clean messy text data with various string issues.

In [ ]:
# Create messy text data
customer_reviews = pd.DataFrame({
    'review_id': range(1, 11),
    'customer_name': ['  John Smith  ', 'jane DOE', 'BOB WILSON', 'Alice Brown',
                     'charlie  davis', 'Emma   Johnson', 'DAVID LEE', '  Sarah Miller',
                     'Michael Chen  ', '  Lisa   Garcia  '],
    'email': ['JOHN@EMAIL.COM', 'Jane@Email.com', '  bob@email.com  ', 'alice@EMAIL.com',
             'Charlie@email.COM', '  emma@email.com', 'david@email.com  ', 'Sarah@EMAIL.COM',
             'michael@email.com', '  LISA@EMAIL.COM  '],
    'phone': ['(555) 123-4567', '555-567-8901', '5559012345', '(555)345-6789',
             '555 789 0123', '555.246.8135', '(555)-135-7902', '555 864 2097',
             '(555)975-3108', '555-310-8642'],
    'product': ['iPhone 12', 'iphone 12', 'IPHONE 12', 'Samsung Galaxy',
               'samsung galaxy', 'MacBook Pro', 'macbook pro', 'iPad Air',
               'ipad air', 'AirPods Pro']
})

print("Messy Text Data:")
print(customer_reviews)

In [ ]:
# Identify string issues
print("=" * 60)
print("STRING ISSUES IDENTIFIED")
print("=" * 60)

# Check for leading/trailing whitespace
has_leading = customer_reviews['customer_name'].str.startswith(' ').sum()
has_trailing = customer_reviews['customer_name'].str.endswith(' ').sum()
print(f"\n1. Whitespace Issues:")
print(f"   Names with leading whitespace: {has_leading}")
print(f"   Names with trailing whitespace: {has_trailing}")

# Case inconsistency
print(f"\n2. Case Inconsistency:")
print(f"   Unique emails (case-sensitive): {customer_reviews['email'].nunique()}")
print(f"   Unique emails (case-insensitive): {customer_reviews['email'].str.lower().nunique()}")

# Product name variations
print(f"\n3. Product Name Variations:")
print(f"   Unique products: {customer_reviews['product'].nunique()}")
print(f"   Values: {customer_reviews['product'].unique()}")

# Phone format variations
print(f"\n4. Phone Format Variations:")
print(f"   Formats found:")
for phone in customer_reviews['phone'].head():
    print(f"   - {phone}")

In [ ]:
# Clean string data
reviews_clean = customer_reviews.copy()

# 1. Clean customer names
# Remove extra whitespace and use title case
reviews_clean['customer_name'] = (reviews_clean['customer_name']
                                  .str.strip()  # Remove leading/trailing
                                  .str.replace(r'\s+', ' ', regex=True)  # Multiple spaces to single
                                  .str.title())  # Title case

print("Cleaned Names:")
print(reviews_clean['customer_name'].tolist())

# 2. Clean emails (lowercase, strip whitespace)
reviews_clean['email'] = (reviews_clean['email']
                         .str.strip()
                         .str.lower())

print("\nCleaned Emails:")
print(reviews_clean['email'].tolist())

# 3. Standardize phone numbers to (XXX) XXX-XXXX format
def standardize_phone(phone):
    """Extract digits and format as (XXX) XXX-XXXX"""
    if pd.isna(phone):
        return None
    # Extract only digits
    digits = re.sub(r'\D', '', str(phone))
    # Format if 10 digits
    if len(digits) == 10:
        return f'({digits[:3]}) {digits[3:6]}-{digits[6:]}'
    return phone  # Return original if can't format

reviews_clean['phone'] = reviews_clean['phone'].apply(standardize_phone)

print("\nStandardized Phone Numbers:")
print(reviews_clean['phone'].tolist())

# 4. Standardize product names
reviews_clean['product'] = reviews_clean['product'].str.title()

print("\nStandardized Products:")
print(reviews_clean['product'].unique())

print("\n✅ All strings cleaned and standardized!")

In [ ]:
# Compare before and after
comparison = pd.DataFrame({
    'Original Name': customer_reviews['customer_name'],
    'Cleaned Name': reviews_clean['customer_name'],
    'Original Email': customer_reviews['email'],
    'Cleaned Email': reviews_clean['email'],
    'Original Phone': customer_reviews['phone'],
    'Cleaned Phone': reviews_clean['phone']
})

print("Before and After Comparison:")
print(comparison)

---

## Part 6: Complete Data Cleaning Pipeline

Let's build a comprehensive cleaning pipeline for a messy dataset.

In [ ]:
# Create comprehensive messy dataset
np.random.seed(42)

messy_data = pd.DataFrame({
    'CUSTOMER ID': [1001, 1002, 1003, 1001, 1004, 1005, 1006, 1007, 1008, 1009, 1010],
    'Customer Name': ['  John Smith  ', 'jane doe', 'BOB WILSON', 'John Smith',
                     'Alice  Brown', 'Charlie Davis', 'Emma   Johnson', 'David Lee',
                     'Sarah Miller', 'Michael Chen', 'Lisa Garcia'],
    'AGE': ['25', '32', 'unknown', '26', '45', '38', '29', '52', '41', '36', '-5'],
    'Purchase Date': ['2024-01-15', '01/20/2024', 'Jan 25, 2024', '2024-02-01',
                     '02/10/2024', 'Feb 15, 2024', '2024-03-01', '03/05/2024',
                     'Mar 10, 2024', '2024-03-15', '03/20/2024'],
    'Product Category': ['Electronics', 'electronics', 'ELECTRONICS', 'Electronics ',
                        'clothing', 'Clothing', 'CLOTHING', 'Home', 'home', 'Electronics', 'Clothing'],
    'Amount': ['$299.99', '$149.50', '$599.00', '$399.99', '$79.99',
              '$129.99', '$89.50', '$249.99', '$199.99', '$449.99', '$159.99'],
    'Status': ['Active', 'active', 'A', 'Active', 'Inactive', 'I', 'Active',
              'ACTIVE', 'Inactive', 'Active', 'active']
})

print("MESSY DATASET - BEFORE CLEANING")
print("=" * 80)
print(messy_data)
print("\nData Types:")
print(messy_data.dtypes)
print("\nData Info:")
messy_data.info()

In [ ]:
# Build comprehensive cleaning function
def clean_customer_data(df):
    """
    Comprehensive data cleaning pipeline
    
    Steps:
    1. Clean column names
    2. Remove duplicates
    3. Clean text fields
    4. Fix data types
    5. Parse dates
    6. Standardize categories
    7. Handle invalid values
    8. Validate results
    """
    print("Starting data cleaning pipeline...\n")
    df_clean = df.copy()
    
    # Step 1: Clean column names
    print("Step 1: Cleaning column names")
    df_clean.columns = (df_clean.columns
                       .str.strip()
                       .str.lower()
                       .str.replace(' ', '_'))
    print(f"  Columns: {list(df_clean.columns)}")
    
    # Step 2: Remove duplicates
    print("\nStep 2: Removing duplicates")
    before = len(df_clean)
    # Sort by purchase_date descending to keep most recent
    df_clean['purchase_date'] = pd.to_datetime(df_clean['purchase_date'], errors='coerce')
    df_clean = df_clean.sort_values('purchase_date', ascending=False)
    df_clean = df_clean.drop_duplicates(subset=['customer_id'], keep='first')
    after = len(df_clean)
    print(f"  Removed {before - after} duplicate records")
    
    # Step 3: Clean text fields
    print("\nStep 3: Cleaning text fields")
    df_clean['customer_name'] = (df_clean['customer_name']
                                 .str.strip()
                                 .str.replace(r'\s+', ' ', regex=True)
                                 .str.title())
    print("  Cleaned customer names")
    
    # Step 4: Fix data types
    print("\nStep 4: Converting data types")
    
    # Clean and convert age
    df_clean['age'] = df_clean['age'].replace('unknown', np.nan)
    df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
    # Set invalid ages (negative or > 120) to NaN
    df_clean.loc[(df_clean['age'] < 0) | (df_clean['age'] > 120), 'age'] = np.nan
    print("  Converted age to numeric")
    
    # Clean and convert amount
    df_clean['amount'] = (df_clean['amount']
                         .str.replace('$', '')
                         .str.replace(',', ''))
    df_clean['amount'] = pd.to_numeric(df_clean['amount'], errors='coerce')
    print("  Converted amount to float")
    
    # Step 5: Parse dates (already done in step 2)
    print("\nStep 5: Date parsing")
    print("  Dates already converted to datetime")
    
    # Step 6: Standardize categories
    print("\nStep 6: Standardizing categories")
    
    # Product category
    df_clean['product_category'] = (df_clean['product_category']
                                    .str.strip()
                                    .str.title())
    print(f"  Product categories: {df_clean['product_category'].unique()}")
    
    # Status
    status_mapping = {
        'A': 'Active',
        'I': 'Inactive',
        'active': 'Active',
        'ACTIVE': 'Active',
        'inactive': 'Inactive',
        'INACTIVE': 'Inactive'
    }
    df_clean['status'] = df_clean['status'].replace(status_mapping)
    print(f"  Status values: {df_clean['status'].unique()}")
    
    # Step 7: Add derived columns
    print("\nStep 7: Adding derived columns")
    df_clean['year'] = df_clean['purchase_date'].dt.year
    df_clean['month'] = df_clean['purchase_date'].dt.month
    df_clean['quarter'] = df_clean['purchase_date'].dt.quarter
    print("  Added year, month, quarter columns")
    
    # Step 8: Final sorting
    df_clean = df_clean.sort_values(['customer_id', 'purchase_date'])
    df_clean = df_clean.reset_index(drop=True)
    
    print("\n✅ Data cleaning pipeline complete!")
    return df_clean

# Apply cleaning pipeline
print("="*80)
data_cleaned = clean_customer_data(messy_data)
print("="*80)

In [ ]:
# Display cleaned data
print("\nCLEANED DATASET")
print("=" * 80)
print(data_cleaned)
print("\nCleaned Data Types:")
print(data_cleaned.dtypes)

In [ ]:
# Validation report
print("\n" + "="*80)
print("CLEANING VALIDATION REPORT")
print("="*80)

print(f"\n1. Shape:")
print(f"   Before: {messy_data.shape}")
print(f"   After:  {data_cleaned.shape}")

print(f"\n2. Duplicates:")
print(f"   Before: {messy_data.duplicated(subset=['CUSTOMER ID']).sum()}")
print(f"   After:  {data_cleaned.duplicated(subset=['customer_id']).sum()}")

print(f"\n3. Missing Values:")
print(f"   Before: {messy_data.isna().sum().sum()}")
print(f"   After:  {data_cleaned.isna().sum().sum()}")
if data_cleaned.isna().sum().sum() > 0:
    print("\n   Missing by column:")
    missing = data_cleaned.isna().sum()
    print(missing[missing > 0])

print(f"\n4. Product Categories:")
print(f"   Before: {messy_data['Product Category'].nunique()} unique values")
print(f"   After:  {data_cleaned['product_category'].nunique()} unique values")
print(f"   Categories: {data_cleaned['product_category'].unique()}")

print(f"\n5. Status Values:")
print(f"   Before: {messy_data['Status'].nunique()} unique values")
print(f"   After:  {data_cleaned['status'].nunique()} unique values")
print(f"   Values: {data_cleaned['status'].unique()}")

print(f"\n6. Data Quality Metrics:")
total_cells = data_cleaned.size
filled_cells = data_cleaned.count().sum()
completeness = (filled_cells / total_cells * 100)
print(f"   Completeness: {completeness:.2f}%")
print(f"   Valid ages: {((data_cleaned['age'] >= 0) & (data_cleaned['age'] <= 120)).sum()} out of {data_cleaned['age'].notna().sum()}")

print("\n" + "="*80)

In [ ]:
# Visualize cleaning impact
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Categories before and after
messy_data['Product Category'].value_counts().plot(kind='bar', ax=axes[0, 0], color='salmon', alpha=0.7)
axes[0, 0].set_title('Product Categories - Before', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Count')
axes[0, 0].tick_params(axis='x', rotation=45)

data_cleaned['product_category'].value_counts().plot(kind='bar', ax=axes[0, 1], color='lightgreen', alpha=0.7)
axes[0, 1].set_title('Product Categories - After', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# 2. Status before and after
messy_data['Status'].value_counts().plot(kind='bar', ax=axes[1, 0], color='salmon', alpha=0.7)
axes[1, 0].set_title('Status - Before', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=45)

data_cleaned['status'].value_counts().plot(kind='bar', ax=axes[1, 1], color='lightgreen', alpha=0.7)
axes[1, 1].set_title('Status - After', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Count')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("✅ Data cleaning visualization complete!")

In [ ]:
# Now perform analysis on cleaned data
print("\nANALYSIS ON CLEANED DATA")
print("="*80)

print("\n1. Summary Statistics:")
print(data_cleaned[['age', 'amount']].describe())

print("\n2. Sales by Category:")
category_sales = data_cleaned.groupby('product_category').agg({
    'amount': ['sum', 'mean', 'count']
}).round(2)
print(category_sales)

print("\n3. Monthly Trends:")
monthly = data_cleaned.groupby('month')['amount'].agg(['sum', 'count']).round(2)
print(monthly)

print("\n4. Customer Status Distribution:")
print(data_cleaned['status'].value_counts())

print("\n✅ Analysis complete! Clean data enables accurate insights.")

---

## Part 7: Data Quality Assessment

Let's create a comprehensive quality assessment function.

In [ ]:
def assess_data_quality(df, name="Dataset"):
    """
    Comprehensive data quality assessment
    """
    print("="*80)
    print(f"DATA QUALITY REPORT: {name}")
    print("="*80)
    
    # 1. Basic Info
    print(f"\n1. BASIC INFORMATION")
    print(f"   Shape: {df.shape}")
    print(f"   Rows: {len(df):,}")
    print(f"   Columns: {len(df.columns)}")
    print(f"   Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # 2. Completeness
    print(f"\n2. COMPLETENESS")
    total_cells = df.size
    missing_cells = df.isna().sum().sum()
    completeness = (total_cells - missing_cells) / total_cells * 100
    print(f"   Total cells: {total_cells:,}")
    print(f"   Missing cells: {missing_cells:,}")
    print(f"   Completeness: {completeness:.2f}%")
    
    if missing_cells > 0:
        print(f"\n   Missing values by column:")
        missing_by_col = df.isna().sum()
        missing_by_col = missing_by_col[missing_by_col > 0].sort_values(ascending=False)
        for col, count in missing_by_col.items():
            pct = count / len(df) * 100
            print(f"     {col}: {count} ({pct:.1f}%)")
    
    # 3. Duplicates
    print(f"\n3. DUPLICATES")
    duplicate_count = df.duplicated().sum()
    print(f"   Exact duplicate rows: {duplicate_count}")
    if duplicate_count > 0:
        pct = duplicate_count / len(df) * 100
        print(f"   Percentage: {pct:.2f}%")
    
    # 4. Data Types
    print(f"\n4. DATA TYPES")
    type_counts = df.dtypes.value_counts()
    for dtype, count in type_counts.items():
        print(f"   {dtype}: {count} columns")
    
    # 5. Cardinality
    print(f"\n5. CARDINALITY (Unique Values)")
    for col in df.columns:
        unique_count = df[col].nunique()
        unique_pct = unique_count / len(df) * 100
        print(f"   {col}: {unique_count} ({unique_pct:.1f}%)")
    
    # 6. Numeric Ranges
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\n6. NUMERIC RANGES")
        for col in numeric_cols:
            print(f"\n   {col}:")
            print(f"     Min: {df[col].min()}")
            print(f"     Max: {df[col].max()}")
            print(f"     Mean: {df[col].mean():.2f}")
            print(f"     Median: {df[col].median()}")
            print(f"     Std Dev: {df[col].std():.2f}")
    
    # 7. Quality Score
    print(f"\n7. OVERALL QUALITY SCORE")
    
    # Completeness score
    completeness_score = completeness
    
    # Uniqueness score (no duplicates = 100)
    uniqueness_score = ((len(df) - duplicate_count) / len(df) * 100) if len(df) > 0 else 0
    
    # Overall score (average)
    overall_score = (completeness_score + uniqueness_score) / 2
    
    print(f"   Completeness: {completeness_score:.2f}%")
    print(f"   Uniqueness: {uniqueness_score:.2f}%")
    print(f"   Overall Quality: {overall_score:.2f}%")
    
    # Rating
    if overall_score >= 90:
        rating = "Excellent ⭐⭐⭐⭐⭐"
    elif overall_score >= 75:
        rating = "Good ⭐⭐⭐⭐"
    elif overall_score >= 60:
        rating = "Fair ⭐⭐⭐"
    else:
        rating = "Poor ⭐⭐"
    
    print(f"   Rating: {rating}")
    
    print("\n" + "="*80)

# Test on messy data
assess_data_quality(messy_data, "Messy Dataset (Before Cleaning)")

In [ ]:
# Test on cleaned data
assess_data_quality(data_cleaned, "Cleaned Dataset (After Cleaning)")

---

## 🎯 Key Takeaways

### What We Learned:

1. **Removing Duplicates**
   - Detect with `duplicated()`
   - Remove with `drop_duplicates()`
   - Keep best record, not just first

2. **Fixing Data Types**
   - Use `pd.to_numeric()` with `errors='coerce'`
   - Clean strings before converting (remove $, commas)
   - Validate after conversion

3. **Parsing Dates**
   - Use `pd.to_datetime()` for conversion
   - Extract components with `.dt` accessor
   - Handle multiple formats automatically

4. **String Cleaning**
   - Strip whitespace with `.str.strip()`
   - Standardize case with `.str.lower()` / `.str.title()`
   - Use regex for complex cleaning

5. **Standardizing Labels**
   - Create mapping dictionaries
   - Use `replace()` or `map()`
   - Reduce cardinality

6. **Building Pipelines**
   - Clean systematically
   - Document each step
   - Validate after cleaning

### Best Practices Recap:

✅ Always preserve original data  
✅ Document cleaning steps  
✅ Validate after each operation  
✅ Handle errors gracefully  
✅ Make cleaning reproducible  
✅ Test on samples first  

---

## 📚 Next Steps

Now proceed to **10_exercises.ipynb** to practice cleaning messy datasets!

**Remember:** Clean data is the foundation of good analysis! 🧹✨